In [1]:
# Initialize the libs and download the dataset
!pip3 install mne -q
!pip3 install awscli
!aws s3 sync --no-sign-request s3://openneuro.org/ds003626 ds003626/

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-auth-oauthlib 1.0.0 requires google-auth>=2.15.0, but you have google-auth 1.4.2 which is incompatible.
botocore 1.29.137 requires urllib3<1.27,>=1.25.4, but you have urllib3 1.24.3 which is incompatible.


Defaulting to user installation because normal site-packages is not writeable
  Using cached urllib3-1.26.15-py2.py3-none-any.whl (140 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.21.0 requires urllib3<1.25,>=1.21.1, but you have urllib3 1.26.15 which is incompatible.
google-auth-oauthlib 1.0.0 requires google-auth>=2.15.0, but you have google-auth 1.4.2 which is incompatible.


^C


In [2]:
import mne
import warnings
import numpy as np
import pandas as pd

np.random.seed(23)

mne.set_log_level(verbose='warning') #to avoid info at terminal
warnings.filterwarnings(action = "ignore", category = DeprecationWarning )
warnings.filterwarnings(action = "ignore", category = FutureWarning )

In [3]:
# The root dir
root_dir = "C:/Users/hazem/Downloads/ds003626"

# Sampling rate
fs = 256

# Select the useful par of each trial. Time in seconds
t_start = 1.5
t_end = 3.5

In [4]:
from au.pre_process import get_subjects_data_and_label

condition = "Inner"

data, labels = get_subjects_data_and_label(root_dir, condition, t_start = t_start, t_end = t_end, fs = fs)

In [5]:
print("Number of subjects", len(data))
print("Data shape: [trials x channels x samples]")
print("Shape", data[0].shape) # Trials, channels, samples

print("Labels")
print(len(labels)) # Time stamp, class , condition, session

Number of subjects 10
Data shape: [trials x channels x samples]
Shape (200, 128, 512)
Labels
10


In [6]:
data_array=np.vstack(data)
label_array=np.hstack(labels)
print(data_array.shape, label_array.shape)

(2236, 128, 512) (2236,)


In [7]:
# Define all the features
from scipy import stats

def mean(x):
    return np.mean(x, axis=-1)

def std(x):
    return np.std(x, axis=-1)

def ptp(x):
    return np.ptp(x, axis=-1)

def var(x):
    return np.var(x, axis=-1)

def minim(x):
    return np.min(x, axis=-1)

def maxim(x):
    return np.max(x, axis=-1)

def argminim(x):
    return np. argmin(x, axis=-1)

def argmaxim(x):
    return np.argmax(x,axis=-1)

def rms(x):
    return np.sqrt(np.mean(x**2, axis=-1))

def abs_diff_signal(x):
    return np.sum(np.abs(np.diff(x, axis=-1)), axis=-1)

def skewness(x):
    return stats.skew(x, axis=-1)

def kurtosis(x):
    return stats.kurtosis(x, axis=-1)

def concatenate_features(x):
    return np.concatenate((mean(x), std(x), ptp(x), var(x), minim(x), maxim(x), argminim(x),
                          argmaxim(x), rms(x), abs_diff_signal(x), skewness(x), kurtosis(x)), axis=-1)

In [8]:
features=[]
for d in data_array:
    features.append(concatenate_features(d))

In [9]:
features_array=np.array(features)
features_array.shape # 1536 / 128 = 12 features

(2236, 1536)

In [10]:
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import metrics, model_selection
from sklearn.metrics import roc_auc_score, recall_score, precision_score, roc_curve
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

# This section contains the function to support the model evaluation


def run_cross_validation(classifier, x_tr, y_tr):
    k_fold = model_selection.KFold(n_splits=10)
    results = model_selection.cross_val_score(classifier, x_tr, y_tr, cv=k_fold, scoring='accuracy')
    print('{:<50} {:.4f}'.format("Cross validation average accuracy with 10-fold:", (results.mean())))

def run_accuracy(y_tst, y_p):
    print('{:<50} {:.4f}'.format("Accuracy", (metrics.accuracy_score(y_tst, y_p))))

def plot_confusion_matrix(y_tst, y_pred, y_labels):
    lbs = y_labels.unique()
    confusion_matrix = metrics.confusion_matrix(y_tst, y_pred)
    matrix_df = pd.DataFrame(confusion_matrix)
    ax = plt.axes()
    sns.set(font_scale=1.3)
    plt.figure(figsize=(10,7))
    sns.heatmap(matrix_df, annot=True, fmt="g", ax=ax, cmap="magma")
    ax.set_title('Confusion Matrix')
    ax.set_xlabel("Predicted label", fontsize =15)
    ax.set_ylabel("True Label", fontsize=15)
    ax.set_yticklabels(list(lbs), rotation = 0)
    plt.show()

# Extract importance
def print_importance(classifier, x_tr):
    importance = pd.DataFrame({'feature': x_tr.columns, 'importance' : np.round(classifier.feature_importances_, 3)})
    importance.sort_values('importance', ascending=False, inplace = True)
    print(importance)

In [11]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

def run_random_forest():
    print("Random Forest")
    # Split dataset into training set and test set
    x_train, x_test, y_train, y_test = train_test_split(features_array, label_array, test_size=0.30) # 70% test and 30% training
    clf = RandomForestClassifier(n_estimators=100)

    # Train the model
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)

    run_accuracy(y_test, y_pred)
    run_cross_validation(clf, x_train, y_train)
    print(classification_report(y_test, y_pred))
    # evaluate_model(clf, x_train, x_test, y_train, y_test)

run_random_forest()

Random Forest
Accuracy                                           0.2668
Cross validation average accuracy with 10-fold:    0.2441
              precision    recall  f1-score   support

           0       0.26      0.21      0.23       183
           1       0.32      0.32      0.32       169
           2       0.23      0.30      0.26       156
           3       0.26      0.25      0.25       163

    accuracy                           0.27       671
   macro avg       0.27      0.27      0.27       671
weighted avg       0.27      0.27      0.27       671



NameError: name 'evaluate_model' is not defined

In [12]:
# Neural Network
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

def run_neural_network():
    print("Neural Network")
    # Split dataset into training set and test set
    x_train, x_test, y_train, y_test = train_test_split(features_array, label_array, test_size=0.20) # 80% test and 20% training
    clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

    # Train the model
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)

    run_accuracy(y_test, y_pred)
    run_cross_validation(clf, x_train, y_train)
    print(classification_report(y_test, y_pred))
    #plot_confusion_matrix(y_test, y_pred, label_array)

run_neural_network()

Neural Network
Accuracy                                           0.2277
Cross validation average accuracy with 10-fold:    0.2388
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       137
           1       0.00      0.00      0.00       103
           2       0.00      0.00      0.00       106
           3       0.23      1.00      0.37       102

    accuracy                           0.23       448
   macro avg       0.06      0.25      0.09       448
weighted avg       0.05      0.23      0.08       448



C:\Users\hazem\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hazem\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hazem\AppData\Roaming\Python\Python310\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

In [13]:
# Multiclass Random Forest
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier

def run_random_forest_multi_class():
    print("Random Forest")
    # Split dataset into training set and test set
    x_train, x_test, y_train, y_test = train_test_split(features_array, label_array, test_size=0.20) # 80% test and 20% training
    rf = RandomForestClassifier(random_state=42, max_features='auto', n_estimators= 200, max_depth=8, criterion='gini')
    clf = OneVsRestClassifier(rf)

    # Train the model
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)

    run_accuracy(y_test, y_pred)
    run_cross_validation(clf, x_train, y_train)
    print(classification_report(y_test, y_pred))
    #print_importance(clf, x_train)
    #plot_confusion_matrix(y_test, y_pred)

run_random_forest_multi_class()

Random Forest
Accuracy                                           0.1964
Cross validation average accuracy with 10-fold:    0.2075
              precision    recall  f1-score   support

           0       0.17      0.12      0.15       112
           1       0.32      0.17      0.22       134
           2       0.19      0.25      0.22       102
           3       0.16      0.26      0.20       100

    accuracy                           0.20       448
   macro avg       0.21      0.20      0.19       448
weighted avg       0.22      0.20      0.20       448



In [14]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

def run_random_forest(X, y):
    print("Random Forest")
    # Split dataset into training set and test set
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20) # 80% test and 20% training
    clf = RandomForestClassifier(random_state=42, max_features='auto', n_estimators= 200, max_depth=8, criterion='gini')

    # Train the model
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)

    run_accuracy(y_test, y_pred)
    run_cross_validation(clf, x_train, y_train)
    print(classification_report(y_test, y_pred))
    #print_importance(clf, x_train)
    #plot_confusion_matrix(y_test, y_pred)

run_random_forest(features_array, label_array)

Random Forest
Accuracy                                           0.2388
Cross validation average accuracy with 10-fold:    0.2372
              precision    recall  f1-score   support

           0       0.28      0.28      0.28       114
           1       0.22      0.27      0.25       103
           2       0.22      0.17      0.19       121
           3       0.23      0.25      0.24       110

    accuracy                           0.24       448
   macro avg       0.24      0.24      0.24       448
weighted avg       0.24      0.24      0.24       448



In [15]:
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split

def run_linear_svc_multi_class():
    print("Linear SVC")
    # Split dataset into training set and test set
    x_train, x_test, y_train, y_test = train_test_split(features_array, label_array, test_size=0.30) # 70% test and 30% training
    rf = LinearSVC(random_state=0, max_iter=10000)
    clf = OneVsRestClassifier(rf)

    # Train the model
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)

    run_accuracy(y_test, y_pred)
    run_cross_validation(clf, x_train, y_train)
    print(classification_report(y_test, y_pred))

run_linear_svc_multi_class()

Linear SVC


C:\Users\hazem\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\hazem\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\hazem\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\hazem\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Accuracy                                           0.2444


C:\Users\hazem\AppData\Roaming\Python\Python310\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
